## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2021.

# ECBM E4040 - Assignment 2- Task 5: Kaggle Open-ended Competition

Kaggle is a platform for predictive modelling and analytics competitions in which companies and researchers post data and statisticians and data miners compete to produce the best models for predicting and describing the data.

If you don't have a Kaggle account, feel free to join at [www.kaggle.com](https://www.kaggle.com). To let the CAs do the grading more conveniently, please __use Lionmail to join Kaggle__ and __use UNI as your username__.

The competition is located here: https://www.kaggle.com/c/ecbm4040-assignment-2-task-5/overview.

You can find detailed description about this in-class competition on the website above. Please read carefully and follow the instructions.

<span style="color:red">__TODO__:</span>
1. Train a custom model for the bottle dataset classification problem. You are free to use any methods taught in the class or found by yourself on the Internet (ALWAYS provide reference to the source).
General training methods include:
    * Dropout
    * Batch normalization
    * Early stopping
    * l1-norm & l2-norm penalization
    
2. You are given the test set to generate your predictions (70% public + 30% private, but you don't know which ones are public/private). Students should achieve an accuracy on the public test set of at least 70%. Two points will be deducted for each 1% below 70% accuracy threshold (i.e. 65% accuracy will have 10 points deducted). The accuracy will be shown on the public leaderboard once you submit your prediction .csv file. The private leaderboard will be released after the competition. The final ranking is based on the private leaderboard result, not the public leaderboard.
3. 

    * Report your results on the Kaggle, for comparison with other students' optimal results (you can do this several times). 
    * Save your best model, using Github Classroom, at the same time when you submit the homework files into Courseworks. See instructions below. 

__Hint__: You can start from what you implemented in task 4. Another classic classification model named 'VGG16' can also be easily implemented. Students are allowed to use pretrained networks, and utilize transfer learning. 

## HW Submission Details:
There are three components to reporting the results of this task: 

**(A) Submission (up to 20 submissions each day) of the .csv prediction file throught the Kaggle platform;**. You should start doing this __VERY early__, so that students can compare their work as they are making progress with model optimization.

**(B) Editing and submitting the content of this Jupyter notebook, through Courseworks; **
(i) The code for your CNN model and for the training function. The code should be stored in __./ecbm4040/neuralnets/kaggle.py__;
(ii) Print out your training process and accuracy __within this notebook__;

**(C) Submitting your best CNN model through Github Classroom repo.**

**Description of (C):** 
For this task, we will continue to use Github classroom to save your model for submission. 

<span style="color:red">__Submission content:__ :</span>
(i) In your Assignment 2 submission folder, create a subfolder called __KaggleModel__. Upload your best model with all the data output (for example, __MODEL.data-00000-of-00001, MODEL.meta, MODEL.index__) into the folder. 
(ii) Remember to delete any intermediate results, **we only want your best model. Do not upload any data files**. The instructors will rerun the uploaded best model and verify against the score which you reported on the Kaggle.



## Load Data

In [1]:
#Generate dataset
import os
import pandas as pd
import numpy as np
from PIL import Image


#Load Training images and labels
train_directory = "./data/kaggle_train_128/train_128" #TODO: Enter path for train128 folder (hint: use os.getcwd())
image_list=[]
label_list=[]
for sub_dir in os.listdir(train_directory):
    print("Reading folder {}".format(sub_dir))
    sub_dir_name=os.path.join(train_directory,sub_dir)
    for file in os.listdir(sub_dir_name):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_list.append(np.array(Image.open(os.path.join(sub_dir_name,file))))
            label_list.append(int(sub_dir))
X_train=np.array(image_list)
y_train=np.array(label_list)

#Load Test images
test_directory = "./data/kaggle_test_128/test_128"#TODO: Enter path for test128 folder (hint: use os.getcwd())
test_image_list=[]
test_df = pd.DataFrame([], columns=['Id', 'X'])
print("Reading Test Images")
for file in os.listdir(test_directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        test_df = test_df.append({
            'Id': filename,
            'X': np.array(Image.open(os.path.join(test_directory,file)))
        }, ignore_index=True)
        
test_df['s'] = [int(x.split('.')[0]) for x in test_df['Id']]
test_df = test_df.sort_values(by=['s'])
test_df = test_df.drop(columns=['s'])
X_test = np.stack(test_df['X'])


print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)

Reading folder 1
Reading folder 3
Reading folder 0
Reading folder 2
Reading folder 4
Reading Test Images
Training data shape:  (15000, 128, 128, 3)
Training labels shape:  (15000,)
Test data shape:  (3500, 128, 128, 3)


## Build and Train Your Model Here

In [2]:
# import
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, AveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
# tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
import pathlib 
train_dir = pathlib.Path('./data/kaggle_train_128')
train_dir = train_dir / 'train_128'

In [4]:
# trdata = ImageDataGenerator(rescale=1./255,validation_split=0.2)
# trdata = ImageDataGenerator(
#         rotation_range=10, # rotation
#         width_shift_range=0.2, # horizontal shift
#         height_shift_range=0.2, # vertical shift
#         zoom_range=0.2, # zoom
#         horizontal_flip=True, # horizontal flip
#         validation_split=0.2,
#         brightness_range=[0.2,1.2]) # brightness
trdata = ImageDataGenerator(validation_split=0.2)
train_data = trdata.flow_from_directory(directory=train_dir, target_size=(128,128), subset='training')
valid_data = trdata.flow_from_directory(directory=train_dir, target_size=(128,128), subset='validation')


Found 12000 images belonging to 5 classes.
Found 3000 images belonging to 5 classes.


In [7]:
model = Sequential()
model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
# model.add(Dropout(0.2))
model.add(Dense(units= 5, activation="softmax"))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 256)      

In [10]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit_generator(generator=train_data, 
                           validation_data= valid_data, 
                           validation_steps=10,
                           epochs=15)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
375/375 [==============================] - 47s 124ms/step - loss: 0.1995 - accuracy: 0.9320 - val_loss: 0.4793 - val_accuracy: 0.8344
Epoch 2/15
375/375 [==============================] - 47s 125ms/step - loss: 0.1976 - accuracy: 0.9335 - val_loss: 0.5990 - val_accuracy: 0.8375
Epoch 3/15
375/375 [==============================] - 48s 127ms/step - loss: 0.1537 - accuracy: 0.9453 - val_loss: 0.7140 - val_accuracy: 0.8219
Epoch 4/15
375/375 [==============================] - 48s 127ms/step - loss: 0.1318 - accuracy: 0.9527 - val_loss: 0.5928 - val_accuracy: 0.8656
Epoch 5/15
375/375 [==============================] - 48s 129ms/step - loss: 0.1401 - accuracy: 0.9519 - val_loss: 0.9045 - val_accuracy: 0.8375
Epoch 6/15
375/375 [==============================] - 48s 129ms/step - loss: 0.1096 - accuracy: 0.9605 - val_loss: 0.6857 - val_accuracy: 0.8625
Epoch 7/15
375/375 [==============================] - 48s 129ms/step - loss: 0.1087 - accuracy: 0.9636 - val_loss: 1.0261 - val_ac

## Save your best model

In [11]:
model.save(filepath = "./KaggleModel")

INFO:tensorflow:Assets written to: ./KaggleModel/assets


In [ ]:
# model link is in <https://drive.google.com/file/d/1kCSzf6l0vbKSmR-bTPhjxcX8gxdngDua/view?usp=sharing>
# This is shared to lionmail

In [ ]:
# loaded_model = tf.keras.models.load_model("./KaggleModel")

## Generate .csv file for Kaggle

The following code snippet can be used to generate your prediction .csv file.

NOTE: If your kaggle results are indicating random performance, then it's likely that the indices of your csv predictions are misaligned.

In [12]:
import numpy as np
import csv

test_dir = pathlib.Path('./data/kaggle_test_128')
test_dir = test_dir / 'test_128'
# tsdata = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization=True)
# test_gen = tsdata.flow_from_directory(directory=test_dir,target_size=(128,128))

predicted_values_generated_by_your_model = {}
for image_path in test_dir.glob("*"):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size = (128, 128))
    #image = test_gen.standardize(image)
    image = np.expand_dims(image, 0)
    preds = model.predict(image)
    pred = np.argmax(preds)    
    predicted_values_generated_by_your_model[image_path.name] = pred

In [13]:
# import csv

with open('predicted.csv','w') as csvfile:
    fieldnames = ['Id','label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index in predicted_values_generated_by_your_model:
        filename = str(index)
        label = str(predicted_values_generated_by_your_model[index])
        writer.writerow({'Id': filename, 'label': label})

In [ ]:
!sudo apt-get install tree
!tree ./ >> README.md